<a href="https://colab.research.google.com/github/lucassmacedo/titanic-challenge-classification/blob/master/Titanic_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


*by [Lucas Macedo](https://www.linkedin.com/in/lucassmacedo//)*

---

# Análise dos sobreviventes do desastre Titanic

A idéia desta análise é trazer insights sobre os dados dos sobreviventes do Titanic e criar um modelo que faça a predição dos sobreviventes.


<center><img alt="Mapa do RJ" width="40%" src="https://ichef.bbci.co.uk/news/640/cpsprodpb/15A5F/production/_115017688_c6122844-332e-4516-a812-e56991e9374a.jpg"></center>




### **Obtenção dos Dados**

Os dados usados nesse projeto, foram retirados diretamente do portal de dados aberto do [Kaggle](https://www.kaggle.com/c/titanic/data)  





In [ ]:
# Iniciando a Declaração das Bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
# Iniciando os dados de Treino e Teste
train = pd.read_csv('../content/sample_data/train.csv')
test = pd.read_csv('../content/sample_data/test.csv')

## Preparação e Limpeza dos dados

---




###Dicionário de variáveis

*  **Survived**: Se sobreviveu ou não
*  **Pclass**: Ticket de classe(Status socio-econômico)
*  **Name** = Nome
*  **Sex**: Sexo
*  **Age**: Idade
*  **SibSp**: Número de irmãos/irmãs ou cônjuges abordo
*  **Parch**: Número de pais ou crianças abordo
*  **Ticket**: Número do Ticket
*  **Fare**: Tarifa paga
*  **Cabin**: Número da cabine
*  **Embarked**: Porta de embarque

In [ ]:
# Alterando os nomes das colunas para Português
train.columns = ['id', 'sobrevivu','classe','nome','sexo','idade','sibSp','Parch','ticket','valor','cabine','portaria'];
test.columns = ['id','classe','nome','sexo','idade','sibSp','Parch','ticket','valor','cabine','portaria'];

Como podemos observar o Dataset possui alguns **Missings Values** sendo necessário um pré processamento para limpeza do mesmo.

In [ ]:
train.head()

,id,sobrevivu,classe,nome,sexo,idade,sibSp,Parch,ticket,valor,cabine,portaria
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test.info()
print('_'*40)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        418 non-null    int64  
 1   classe    418 non-null    int64  
 2   nome      418 non-null    object 
 3   sexo      418 non-null    object 
 4   idade     332 non-null    float64
 5   sibSp     418 non-null    int64  
 6   Parch     418 non-null    int64  
 7   ticket    418 non-null    object 
 8   valor     417 non-null    float64
 9   cabine    91 non-null     object 
 10  portaria  418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         891 non-null    int64  
 1   sobrevivu  891 non-null    int64  
 2   classe     891

Percebemos que existem **Missing Values** que precisamos que sejam concertados para que nosso modelo não tenha **out layer**


In [ ]:
train[train['portaria'].isna()]

,id,sobrevivu,classe,nome,sexo,idade,sibSp,Parch,ticket,valor,cabine,portaria
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


Conforme o Gráfico acima os valores acima de 30 são do portão de embarque C  e nossos valores faltantes também tem o valor do ticket de 80, por isso atualziaremos para C o valor da Portaria

In [ ]:
train["portaria"] = train["portaria"].fillna("C")

Agora com os valores da coluna Portaria ajustados, iremos transformalos em números

In [ ]:
portaria = {'S': 1, 'C': 2, 'Q': 3}
train["portaria"] = train["portaria"].map(portaria)
test["portaria"] = test["portaria"].map(portaria)

In [ ]:
train.info()
print('_'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         891 non-null    int64  
 1   sobrevivu  891 non-null    int64  
 2   classe     891 non-null    int64  
 3   nome       891 non-null    object 
 4   sexo       891 non-null    object 
 5   idade      714 non-null    float64
 6   sibSp      891 non-null    int64  
 7   Parch      891 non-null    int64  
 8   ticket     891 non-null    object 
 9   valor      891 non-null    float64
 10  cabine     204 non-null    object 
 11  portaria   891 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        418 non-null    int64  
 1   clas

Lembrando que o Objetivo é deixar os valores totais de cada coluna sendoo mesmo do total de entradas, ou seja sem valores nulos.

Agora vamos para a coluna **Idade**, este campo eu escolhi preencher da seguinte forma: Mulheres = Média de Mulheres, Homens = Média de Homens

In [ ]:
train[train['idade'].isnull()]

,id,sobrevivu,classe,nome,sexo,idade,sibSp,Parch,ticket,valor,cabine,portaria
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,3
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,1
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,2
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,2
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,2
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,1
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,1
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,1


In [ ]:
media_mulheres = train[train['sexo'] == 'female']['idade'].median();
train.loc[train['sexo'] == 'female','idade'] = train.loc[train['sexo'] == 'female','idade'].fillna(media_mulheres);

media_homens = train[train['sexo'] == 'male']['idade'].median();
train.loc[train['sexo'] == 'male','idade'] = train.loc[train['sexo'] == 'male','idade'].fillna(media_homens)


media_mulheres = test[test['sexo'] == 'female']['idade'].median();
test.loc[test['sexo'] == 'female','idade'] = test.loc[test['sexo'] == 'female','idade'].fillna(media_mulheres);

media_homens = test[test['sexo'] == 'male']['idade'].median();
test.loc[test['sexo'] == 'male','idade'] = test.loc[test['sexo'] == 'male','idade'].fillna(media_homens)




In [ ]:
train.info()
print('_'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         891 non-null    int64  
 1   sobrevivu  891 non-null    int64  
 2   classe     891 non-null    int64  
 3   nome       891 non-null    object 
 4   sexo       891 non-null    object 
 5   idade      891 non-null    float64
 6   sibSp      891 non-null    int64  
 7   Parch      891 non-null    int64  
 8   ticket     891 non-null    object 
 9   valor      891 non-null    float64
 10  cabine     204 non-null    object 
 11  portaria   891 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        418 non-null    int64  
 1   clas

Aqui existe um problema. A coluna Cabine tem muitos valores nullos e eu não encontrei uma lógica para preencher os valores faltantes de forma que faça sentido. Por isso eu resolvi cortados fora. A Variavel Ticket também não tem influencia direta no modelo por ser apenas o número da passagem.

In [ ]:
remover = ["cabine", "ticket"]
train = train.drop(remover, axis = 1)
test = test.drop(remover, axis = 1)

In [ ]:
train.info()
print('_'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         891 non-null    int64  
 1   sobrevivu  891 non-null    int64  
 2   classe     891 non-null    int64  
 3   nome       891 non-null    object 
 4   sexo       891 non-null    object 
 5   idade      891 non-null    float64
 6   sibSp      891 non-null    int64  
 7   Parch      891 non-null    int64  
 8   valor      891 non-null    float64
 9   portaria   891 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 69.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        418 non-null    int64  
 1   classe    418 non-null    int64  
 2   nome      418 non-null    object 
 3   sexo   

Agora nos resta apenas uma coluna com valores faltantes que é a coluna Valor do dataset de Test

In [ ]:
test[test['valor'].isnull()]

,id,classe,nome,sexo,idade,sibSp,Parch,valor,portaria
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,NaN,1


Que iremos preencher com a média do preço das passagens do portão de embaque 1

In [ ]:
test[test['valor'].isnull()]

,id,classe,nome,sexo,idade,sibSp,Parch,valor,portaria
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,NaN,1


In [ ]:
valor_medio = test[test['portaria'] == 1]['valor'].mean();

test.loc[test['portaria'] == 1,'valor'] = test.loc[test['portaria'] == 1,'valor'].fillna(valor_medio);



In [ ]:
train.info()
print('_'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         891 non-null    int64  
 1   sobrevivu  891 non-null    int64  
 2   classe     891 non-null    int64  
 3   nome       891 non-null    object 
 4   sexo       891 non-null    object 
 5   idade      891 non-null    float64
 6   sibSp      891 non-null    int64  
 7   Parch      891 non-null    int64  
 8   valor      891 non-null    float64
 9   portaria   891 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 69.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        418 non-null    int64  
 1   classe    418 non-null    int64  
 2   nome      418 non-null    object 
 3   sexo   

Pronto, agora os 2 datasets de teste e treino estão prontos para ser usados.



```
# This is formatted as code
```

##Análise do Dados

Após alguns tratamentos feito na base, começaremos as análises no *dataset*.

<center><img alt="Analise" width="40%" src="https://image.freepik.com/free-vector/flat-design-scientist-holding-dna-molecules-illustration_23-2148536282.jpg"></center>

In [ ]:
test.describe()

NameError: ignored